Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your collaborators below:

In [1]:
COLLABORATORS = "Cyan Bastiaans"

---

## Part A (1 point)

<div class="alert alert-success">
In your own words, explain what a Turing machine is. How is the concept of a Turing machine related to the study of cognition and language? Why is it so important?
</div>

A Turing Machine is an abstract computation system that can simulate any algorithm if given a program for it. It takes in inputs, and uses a set of rules to determine the output. It is related to the study of cognition and language because like language and cognition, it is a formal system. It uses predefined sets of rules and symbols to generate a certain output. It is much like language, because language can be thought of by applying grammatical rules to generate sentences—and cognition—which (more complicatedly) uses a set of rules on incoming stimulus to produce an apt response.

---

In the next few parts of this problem, you will write code for several different Turing machines. To facilitate this, we have provided you the following code which runs a Turing machine, and optionally prints out its state at each step:

In [2]:
def turing_machine(tape, program, verbose=False):
    """Run a given Turing machine with the given input.

    The program, accepted as the second argument, should accept two parameters:

      * state: A string representing the current state of the Turing machine.
        For example, the initial state that the Turing machine starts in is
        a string "initial state".
      * symbol: The current symbol being read on the tape, for example,
        "a" or "b". If there is nothing to read, then the symbol will be an
        empty string.

    The program should return a 3-tuple of (new state, new symbol, action):

      * new state: The next state of the Turing machine, as determined by
        your program.
      * new symbol: A new symbol to write on the tape *before* the Turing
        machine executes the action.
      * action: The action that the Turing machine should take after writing
        the new symbol. This should be a string, and should be either "left"
        (move left), "right" (move right), or "halt" (stop executing).

    The program is always run starting from the beginning (index 0) of the tape.
    If the Turing machine is at index 0, and the program returns an action of
    "left", then the tape will be extended to the left. If the Turing machine is
    at the last index of the tap, and the program returns an action of "right",
    then the tape will be extended to the right. In both cases, the symbol that
    is read after extending the tape is always an empty string.
    
    Parameters
    ----------
    tape: list
        A list of symbols representing the tape
    program: function
        The Turing machine to be run. Should accept two paramters:
        the current state of the Turing machine, and the current symbol
        being read.
    verbose: boolean (optional)
        Whether to print out the current state of the Turing machine
        after each step.
        
    Returns
    -------
    The symbol being read by the Turing machine when it halted.
        
    """
    
    # start in the intial state with the head over the first position
    state = "initial state"
    head = 0
    action = ""
    
    # helper function to print out the current state of the
    # Turing machine
    def print_tape(tape, head, state):
        print("state: " + str(state))
        tape = [str(x) if x != "" else " " for x in tape]
        print(" ".join(tape))
        print((" " * head * 2) + "^")
    
    # don't actually loop forever, to prevent infinite for loops
    for i in range(1000):
        if verbose:
            print_tape(tape, head, state)
        
        # run the next step of the program, which tells us the new
        # state, the new symbol, and what action to take
        state, tape[head], action = program(state, tape[head])
        
        # if the action is left, then make sure there is enough
        # tape on the left size
        if action == "left":
            if head == 0:
                tape.insert(0, "")
            else:
                head -= 1
                
        # if the action is right, then make sure there is enough
        # tape on the right side
        elif action == "right":
            if head == (len(tape) - 1):
                tape.append("")
            head += 1
            
        # if the action is halt, then stop
        elif action == "halt":
            break
            
        # unrecognized action
        else:
            raise ValueError("invalid action: " + str(action))

    if verbose:
        print_tape(tape, head, state)

    # return the last symbol
    if i == 1000:
        return None
    else:
        return tape[head]

As an example of how to write a program for the `turing_machine` function, here is a simple Turing machine that detects if a string consists of all a's (i.e., $a^k$):

In [3]:
def ak_program(state, symbol):
    """A program for a Turing machine, which computes whether a string
    follows the form of a^k.

    Halts on "y" if the string does follow the form a^k, and halts on 
    "n" if it does not.
    
    Parameters
    ----------
    state: string
        The current state of the Turing machine
    symbol: string
        The symbol currently read by the Turing machine
        
    Returns
    -------
    3-tuple of (new state, new symbol, action)
        The new state of the Turing machine, the symbol
        that should be written to the current position, and the
        action that should be taken after writing.
    
    """
    # The first state is always "initial state". If the symbol is 
    # "a", though, then we want to move right and continue looking 
    # for another "a".
    if state == "initial state" and symbol == "a":
        return ("found a", "a", "right")

    # If the state is "found a", then we know there is an "a"
    # directly to the left, so we want to verify that the current
    # symbol is another "a". If it is, then we can continue
    # moving right and looking for "a"s
    elif state == "found a" and symbol == "a":
        return ("found a", "a", "right")

    # If the state is "found a", and there is no current symbol,
    # then we know our string fits the bill.
    elif state == "found a" and symbol == "":
        return ("found a", "y", "halt")

    # If the state/symbol is anything else, then this string does
    # not follow the form of a^k
    else:
        return (state, "n", "halt")

We can verify that this program does the right thing with a few test cases:

In [4]:
# string with all a's
turing_machine(list("aaa"), ak_program, verbose=True)

state: initial state
a a a
^
state: found a
a a a
  ^
state: found a
a a a
    ^
state: found a
a a a  
      ^
state: found a
a a a y
      ^


'y'

In [5]:
# string with some a's and some other symbols
turing_machine(list("aab"), ak_program, verbose=True)

state: initial state
a a b
^
state: found a
a a b
  ^
state: found a
a a b
    ^
state: found a
a a n
    ^


'n'

In [6]:
# string with no a's
turing_machine(list("b"), ak_program, verbose=True)

state: initial state
b
^
state: initial state
n
^


'n'

In [7]:
# no symbols at all
turing_machine(list(" "), ak_program, verbose=True)

state: initial state
 
^
state: initial state
n
^


'n'

---

## Part B (0.5 points)

<div class="alert alert-success">
As a warmup, write code to implement a simple Turing machine that detects whether a string consists of some number of $a$'s followed by some (not necessarily the same) number of $b$'s.
</div>

In [8]:
def akbm_program(state, symbol):
    """A program for a Turing machine, which computes whether a string
    follows the form of a^k b^m.

    You should assume that the machine starts in the state "initial state",
    and that it begins on the leftmost character of the string. Your Turing
    machine should halt on the character "y" if the string does follow the
    form of a^k b^m, and "n" otherwise.

    Hint: you can do this using only three states (including the initial state)
    
    Parameters
    ----------
    state: string
        The current state of the Turing machine
    symbol: string
        The symbol currently read by the Turing machine
        
    Returns
    -------
    3-tuple of (new state, new symbol, action)
        The new state of the Turing machine, the symbol
        that should be written to the current position, and the
        action that should be taken after writing.
    
    """
    
    if state == "initial state":
        if symbol == "":
            return ("a", "n", "halt")
        if symbol == "a":
            return ("a", "a", "right")
        if symbol == "b":
            return ("b", "n", "halt")
    
    if state == "a":
        if symbol == "":
            return ("a", "n", "halt")
        if symbol == "a":
            return ("a", "a", "right")
        elif symbol == "b":
            return ("b", "b", "right")
    
    if state == "b":
        if symbol == "":
            return ("b", "y", "halt")
        if symbol == "a":
            return ("b", "n", "halt")
        elif symbol == "b":
            return ("b", "b", "right")    
        
    else:
        return (state, "n", "halt")

You can use the provided `turing_machine` function to see what your program halts on for a given input. For example:

In [9]:
turing_machine(["b", "a"], akbm_program)

'n'

You may find it useful to try out various inputs using the `verbose` flag, to see what the state of the Turing machine is at each step:

In [10]:
turing_machine(["a", "b"], akbm_program, verbose=True)

state: initial state
a b
^
state: a
a b
  ^
state: b
a b  
    ^
state: b
a b y
    ^


'y'

In [11]:
# add your own test cases here!


In [12]:
"""Test that anbm_program halts on the correct symbols for various inputs."""
from nose.tools import assert_equal, assert_less_equal
assert_equal(turing_machine(list(" "), akbm_program), 'n')
assert_equal(turing_machine(list("ab"), akbm_program), 'y')
assert_equal(turing_machine(list("a"), akbm_program), 'n')
assert_equal(turing_machine(list("b"), akbm_program), 'n')
assert_equal(turing_machine(list("ba"), akbm_program), 'n')
assert_equal(turing_machine(list("aba"), akbm_program), 'n')
assert_equal(turing_machine(list("abb"), akbm_program), 'y')
assert_equal(turing_machine(list("aaabb"), akbm_program), 'y')
assert_equal(turing_machine(list("aabb"), akbm_program), 'y')
assert_equal(turing_machine(list("abab"), akbm_program), 'n')
assert_equal(turing_machine(list("aaaaaabbbb"), akbm_program), 'y')
assert_equal(turing_machine(list("aaaaaabbbbbb"), akbm_program), 'y')
assert_equal(turing_machine(list("aaaaaabbbbbbbb"), akbm_program), 'y')

# check that a finite number of states is used
import turing_machine_helper as tmh
assert_equal(turing_machine(list("a" * 5 + "b" * 5), akbm_program), 'y')
assert_equal(turing_machine(list("a" * 10 + "b" * 10), akbm_program), 'y')
states1 = tmh.turing_machine(list("a" * 5 + "b" * 5), akbm_program, return_states=True)
states2 = tmh.turing_machine(list("a" * 10 + "b" * 10), akbm_program, return_states=True)
assert_equal(states1, states2, "number of states changes with input size")
assert_less_equal(len(states2), 20, "too many states")

print("Success!")

Success!


---

## Part C (1.5 points)

<div class="alert alert-success">
Let's do something a little more difficult. Now, write a program that can determine whether a string is of the form $a^kb^k$. Note that this time the number of $a$'s and $b$'s must the the same.
</div>

<div class="alert alert-warning">
As a hint, recall that a Turing Machine can replace the symbol currently being read with a new value -- this can be useful if you want to mark the location of particular symbols in your input sequence
</div>

In [13]:
def akbk_program(state, symbol):
    """A program for a Turing machine, which computes whether a string
    follows the form of a^k b^k.

    Halts on "y" if the string does follow the form a^k b^k, and halts on 
    "n" if it does not.

    Hint: you can do this in 5 states (including the initial state). You
    may want to introduce a new symbol beyond just "a", "b", "y", and "n".
    
    Parameters
    ----------
    state: string
        The current state of the Turing machine
    symbol: string
        The symbol currently read by the Turing machine
        
    Returns
    -------
    3-tuple of (new state, new symbol, action)
        The new state of the Turing machine, the symbol
        that should be written to the current position, and the
        action that should be taken after writing.
    
    """
    mark_a = "ma"
    mark_b = "mb"
    i = "initial state"

    if state == i:
        if symbol == "a":
            return ("Find B", mark_a, "right")
        else:
            return (i, "n", "halt")
    
    if state == "Find B":
        if symbol == "a":
            return ("Find B", "a", "right")        
        if symbol == "b":
            return ("Find A", mark_b, "left")
        if symbol == "":
            return ("Find B", "n", "halt")
        if symbol == mark_b:
            return ("Find B", mark_b, "right")
        if symbol == mark_a:
            return ("Find B", mark_a, "right")
    
    if state == "Find A":
        if symbol == "b":
            return ("Find A", "b", "left")        
        if symbol == "a":
            return ("Find B", mark_a, "right")
        if symbol == "":
            return ("runthru", "", "right")
        if symbol == mark_b:
            return ("Find A", mark_b, "left")
        if symbol == mark_a:
            return ("Find A", mark_a, "left")
    
    if state == "runthru":
        if symbol == "a" or symbol == "b":
            return ("runthru", "n", "halt")
        elif symbol == mark_a:
            return ("runthru", mark_a, "right")
        elif symbol == mark_b:
            return ("runthru", mark_b, "right")
        else:
            return ("runthru", "y", "halt")
    

As before, you will probably find it useful to play around with different inputs with the `verbose` option set to `True`, to see what the Turing machine is doing at each step:

In [14]:
turing_machine(["a", "a", "b", "b"], akbk_program, verbose=True)

state: initial state
a a b b
^
state: Find B
ma a b b
  ^
state: Find B
ma a b b
    ^
state: Find A
ma a mb b
  ^
state: Find B
ma ma mb b
    ^
state: Find B
ma ma mb b
      ^
state: Find A
ma ma mb mb
    ^
state: Find A
ma ma mb mb
  ^
state: Find A
ma ma mb mb
^
state: Find A
  ma ma mb mb
^
state: runthru
  ma ma mb mb
  ^
state: runthru
  ma ma mb mb
    ^
state: runthru
  ma ma mb mb
      ^
state: runthru
  ma ma mb mb
        ^
state: runthru
  ma ma mb mb  
          ^
state: runthru
  ma ma mb mb y
          ^


'y'

In [15]:
# add your own test cases here!


In [16]:
"""Test that the akbk_program halts on the correct symbols for various inputs."""
from nose.tools import assert_equal, assert_less_equal
assert_equal(turing_machine(list(" "), akbk_program), 'n')
assert_equal(turing_machine(list("ab"), akbk_program), 'y')
assert_equal(turing_machine(list("a"), akbk_program), 'n')
assert_equal(turing_machine(list("b"), akbk_program), 'n')
assert_equal(turing_machine(list("ba"), akbk_program), 'n')
assert_equal(turing_machine(list("aba"), akbk_program), 'n')
assert_equal(turing_machine(list("abb"), akbk_program), 'n')
assert_equal(turing_machine(list("aaabb"), akbk_program), 'n')
assert_equal(turing_machine(list("aabb"), akbk_program), 'y')
assert_equal(turing_machine(list("abab"), akbk_program), 'n')
assert_equal(turing_machine(list("aaaaaabbbb"), akbk_program), 'n')
assert_equal(turing_machine(list("aaaaaabbbbbb"), akbk_program), 'y')
assert_equal(turing_machine(list("aaaaaabbbbbbbb"), akbk_program), 'n')

# check that a finite number of states is used
import turing_machine_helper as tmh
assert_equal(turing_machine(list("a" * 5 + "b" * 5), akbk_program), 'y')
assert_equal(turing_machine(list("a" * 10 + "b" * 10), akbk_program), 'y')
states1 = tmh.turing_machine(list("a" * 5 + "b" * 5), akbk_program, return_states=True)
states2 = tmh.turing_machine(list("a" * 10 + "b" * 10), akbk_program, return_states=True)
assert_equal(states1, states2, "number of states changes with input size")
assert_less_equal(len(states2), 20, "too many states")

print("Success!")

Success!


---

## Part D (2 points)

<div class="alert alert-success">
Based off of your previous Turing machine, construct a new Turing machine that now determines whether a string is of the form $a^kb^kc^k$. Note that now there are <i>three</i> symbols you must account for, and there must be exactly the same number of each of them.
</div>

In [17]:
def akbkck_program(state, symbol):
    """A program for a Turing machine, which computes whether a string
    follows the form of a^k b^k c^k.

    Halts on "y" if the string does follow the form a^k b^k c^k, and halts
    on "n" if it does not.

    Hint: you can do this in 6 states, including the initial state. You may
    want to introduce more symbols beyond just "a", "b", "y", and "n".
    
    Parameters
    ----------
    state: string
        The current state of the Turing machine
    symbol: string
        The symbol currently read by the Turing machine
        
    Returns
    -------
    3-tuple of (new state, new symbol, action)
        The new state of the Turing machine, the symbol
        that should be written to the current position, and the
        action that should be taken after writing.
    
    """
    mark_a = "ma"
    mark_b = "mb"
    mark_c = "mc"
    i = "initial state"

    if state == i:
        if symbol == "a":
            return ("Find B", mark_a, "right")
        else:
            return (i, "n", "halt")
    
    if state == "Find B":
        if symbol == "b":
            return ("Find C", mark_b, "right")
        if symbol == "":
            return ("Find B", "n", "halt")
        if symbol == "c":
            return ("Find B", "n", "halt")
        else:
            return ("Find B", symbol, "right")
        
    if state == "Find C":
        if symbol == "c":
            return ("Find A", mark_c, "left")
        if symbol == "":
            return ("Find C", "n", "halt")
        else:
            return ("Find C", symbol, "right")
        
    if state == "Find A":
        if symbol == "":
            return ("Runthru", symbol, "right")
        if symbol == "a":
            return ("Find B", mark_a, "right")
        if symbol != "a":
            return ("Find A", symbol, "left")
    
    if state == "Runthru":
        if symbol == "":
            return ("Runthru", "y", "halt")
        if (symbol != mark_a) and (symbol != mark_b) and (symbol != mark_c):
            return ("Runthru", "n", "halt")
        if symbol != "":
            return ("Runthru", symbol, "right")
        
        
            
    
    
    

In [18]:
turing_machine(["a", "b", "c"], akbkck_program, verbose=True)

state: initial state
a b c
^
state: Find B
ma b c
  ^
state: Find C
ma mb c
    ^
state: Find A
ma mb mc
  ^
state: Find A
ma mb mc
^
state: Find A
  ma mb mc
^
state: Runthru
  ma mb mc
  ^
state: Runthru
  ma mb mc
    ^
state: Runthru
  ma mb mc
      ^
state: Runthru
  ma mb mc  
        ^
state: Runthru
  ma mb mc y
        ^


'y'

In [19]:
# add your own test cases here!


In [20]:
"""Test that the akbkck_program halts on the correct symbols for various inputs."""
from nose.tools import assert_equal, assert_less_equal
assert_equal(turing_machine(list(" "), akbkck_program), 'n')
assert_equal(turing_machine(list("a"), akbkck_program), 'n')
assert_equal(turing_machine(list("b"), akbkck_program), 'n')
assert_equal(turing_machine(list("c"), akbkck_program), 'n')
assert_equal(turing_machine(list("abc"), akbkck_program), 'y')
assert_equal(turing_machine(list("cba"), akbkck_program), 'n')
assert_equal(turing_machine(list("bca"), akbkck_program), 'n')
assert_equal(turing_machine(list("acb"), akbkck_program), 'n')
assert_equal(turing_machine(list("abcabc"), akbkck_program), 'n')
assert_equal(turing_machine(list("aabc"), akbkck_program), 'n')
assert_equal(turing_machine(list("abbc"), akbkck_program), 'n')
assert_equal(turing_machine(list("abcc"), akbkck_program), 'n')
assert_equal(turing_machine(list("aaaabbbbcccc"), akbkck_program), 'y')
assert_equal(turing_machine(list("aaaabbbbccc"), akbkck_program), 'n')
assert_equal(turing_machine(list("aaabbbbcccc"), akbkck_program), 'n')
assert_equal(turing_machine(list("aaaabbbcccc"), akbkck_program), 'n')
assert_equal(turing_machine(list("aaaaabbbbbccccc"), akbkck_program), 'y')

# check that a finite number of states is used
import turing_machine_helper as tmh
assert_equal(turing_machine(list("a" * 5 + "b" * 5 + "c" * 5), akbkck_program), 'y')
assert_equal(turing_machine(list("a" * 10 + "b" * 10 + "c" * 10), akbkck_program), 'y')
states1 = tmh.turing_machine(list("a" * 5 + "b" * 5 + "c" * 5), akbkck_program, return_states=True)
states2 = tmh.turing_machine(list("a" * 10 + "b" * 10 + "c" * 10), akbkck_program, return_states=True)
assert_equal(states1, states2, "number of states changes with input size")
assert_less_equal(len(states2), 20, "too many states")

print("Success!")

Success!


---
## Part E (1 point)

In this problem, you wrote three Turing machines which could detect strings of the following forms:

* $a^k b^m$
* $a^k b^k$
* $a^k b^k c^k$

<div class="alert alert-success">At what level of the Chomsky hierarchy are each of these languages? How are they related to <i>computable languages</i>, which are the ones that Turing machines can recognize?</div>

$a^k b^m$ is a regular language or type 3. $a^k b^k$ is a context free language or type 2. $a^k b^k c^k$ is a context sensitive language or type 1. They are all related to computable language in the sense that they are each subsets of a computable language and a Turing Machine can recognize them, but computable languages are much more complex and are recursively ennumerable languages—meaning there are no restrictions on the number of rules the language can have.

---

Before turning this problem in remember to do the following steps:

1. **Restart the kernel** (Kernel$\rightarrow$Restart)
2. **Run all cells** (Cell$\rightarrow$Run All)
3. **Save** (File$\rightarrow$Save and Checkpoint)

<div class="alert alert-danger">After you have completed these three steps, ensure that the following cell has printed "No errors". If it has <b>not</b> printed "No errors", then your code has a bug in it and has thrown an error! Make sure you fix this error before turning in your problem set.</div>

In [21]:
print("No errors!")

No errors!
